In [1]:
import tensorflow.keras.backend as K
import numpy as np
import dask as d
import os
import cv2
import json
import multiprocessing
import matplotlib.pyplot as plt

from glob import glob
from copy import deepcopy
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy
from tensorflow.keras.utils import Sequence
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications import resnet50

In [2]:
n_cores = multiprocessing.cpu_count()
n_cores

16

In [3]:
from fl_tissue_model_tools import data_prep, dev_config, models, defs
import fl_tissue_model_tools.preprocessing as prep

In [4]:
dirs = dev_config.get_dev_directories("../dev_paths.txt")

# Set up model training parameters

In [5]:
with open("../model_training/invasion_depth_training_values.json", 'r') as fp:
       training_values = json.load(fp)
training_values["rs_seed"] = None if (training_values["rs_seed"] == "None") else training_values["rs_seed"]

In [6]:
training_values

{'batch_size': 32,
 'frozen_epochs': 50,
 'fine_tune_epochs': 50,
 'val_split': 0.2,
 'early_stopping_patience': 25,
 'early_stopping_min_delta': 0.0001,
 'rs_seed': None}

In [7]:
with open("../model_training/invasion_depth_best_hp.json", 'r') as fp:
    best_hp = json.load(fp)

In [8]:
best_hp

{'adam_beta_1': 0.9066466810625207,
 'adam_beta_2': 0.9947698608592066,
 'fine_tune_lr': 9.141354728081903e-05,
 'frozen_lr': 0.00022767552973325001,
 'last_resnet_layer': 'conv5_block1_out'}

In [20]:
### Training paths ###
root_data_path = f"{dirs.data_dir}/invasion_data/"
model_training_path = f"{dirs.analysis_dir}/resnet50_invasion_model"
demo_model_training_path = f"{model_training_path}/demo"
mcp_best_frozen_weights_file = f"{demo_model_training_path}/best_frozen_weights.h5"
mcp_best_finetune_weights_file = f"{demo_model_training_path}/best_finetune_weights.h5"


### General training parameters ###
resnet_inp_shape = (128, 128, 3)
# Binary classification -> only need 1 output unit
n_outputs = 1
seed = training_values["rs_seed"]
val_split = training_values["val_split"]
batch_size = training_values["batch_size"]
frozen_epochs = training_values["frozen_epochs"]
fine_tune_epochs = training_values["fine_tune_epochs"]
# frozen_epochs = 10
# fine_tune_epochs = 10
adam_beta_1 = best_hp["adam_beta_1"]
adam_beta_2 = best_hp["adam_beta_2"]
frozen_lr = best_hp["frozen_lr"]
fine_tune_lr = best_hp["fine_tune_lr"]
class_labels = {"no_invasion": 0, "invasion": 1}
last_resnet_layer = best_hp["last_resnet_layer"]


### Early stopping ###
es_criterion = "val_loss"
es_mode = "min"
# Update these depending on seriousness of experiment
es_patience = training_values["early_stopping_patience"]
es_min_delta = training_values["early_stopping_min_delta"]


### Model saving ###
mcp_criterion = "val_loss"
mcp_mode = "min"
mcp_best_only = True
# Need to set to True otherwise base model "layer" won't save/load properly
mcp_weights_only = True

In [10]:
data_prep.make_dir(demo_model_training_path)

# Prep for loading data

In [11]:
rs = np.random.RandomState(seed)

In [12]:
# Training & validation data (drawn from same image set & randomly assigned)
tv_class_paths = {v: glob(f"{root_data_path}/train/{k}/*.tif") for k, v in class_labels.items()}
for k, v in tv_class_paths.items():
    rs.shuffle(v)

In [13]:
train_data_paths, val_data_paths = data_prep.get_train_val_split(tv_class_paths, val_split=val_split)

# Datasets

In [14]:
train_datagen = data_prep.InvasionDataGenerator(
    train_data_paths,
    class_labels,
    batch_size,
    resnet_inp_shape[:2],
    rs,
    class_weights=True,
    shuffle=True,
    augmentation_function=prep.augment_imgs
)

In [15]:
train_datagen.class_counts

{0: 407, 1: 109}

In [16]:
train_datagen.class_weights

{0: 0.6339066339066339, 1: 2.36697247706422}

In [17]:
val_datagen = data_prep.InvasionDataGenerator(
    val_data_paths,
    class_labels,
    batch_size,
    resnet_inp_shape[:2],
    rs,
    class_weights=train_datagen.class_weights,
    shuffle=True,
    augmentation_function=train_datagen.augmentation_function
)

In [18]:
val_datagen.class_counts

{0: 102, 1: 27}

# Model

In [19]:
K.clear_session()

In [21]:
tl_model = models.build_ResNet50_TL(
    n_outputs,
    resnet_inp_shape,
    base_last_layer=last_resnet_layer,
    # Switch to softmax once n_outputs > 1
    output_act="sigmoid",
    base_model_trainable=False
)

# Frozen training

In [22]:
tl_model.compile(
    optimizer=Adam(learning_rate=frozen_lr, beta_1=adam_beta_1, beta_2=adam_beta_2),
    loss=BinaryCrossentropy(),
    weighted_metrics=[BinaryAccuracy()]
)

In [23]:
tl_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
base_model (Functional)      (None, 4, 4, 2048)        14644096  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 14,646,145
Trainable params: 2,049
Non-trainable params: 14,644,096
_________________________________________________________________


In [24]:
# tl_model.get_layer("base_model").summary()

In [25]:
es_callback = EarlyStopping(monitor=es_criterion, mode=es_mode, min_delta=es_min_delta, patience=es_patience)
mcp_callback = ModelCheckpoint(mcp_best_frozen_weights_file, monitor=mcp_criterion, mode=mcp_mode, save_best_only=mcp_best_only, save_weights_only=mcp_weights_only)

In [26]:
h1 = tl_model.fit(
    train_datagen,
    validation_data=val_datagen,
    epochs=frozen_epochs,
    callbacks=[es_callback, mcp_callback],
    workers=n_cores
)

Epoch 1/50
16/16 [==============================] - 16s 445ms/step - loss: 0.7761 - binary_accuracy: 0.4932 - val_loss: 0.6945 - val_binary_accuracy: 0.5411
Epoch 2/50
16/16 [==============================] - 5s 231ms/step - loss: 0.6366 - binary_accuracy: 0.6311 - val_loss: 0.6052 - val_binary_accuracy: 0.6924
Epoch 3/50
16/16 [==============================] - 5s 204ms/step - loss: 0.5424 - binary_accuracy: 0.7574 - val_loss: 0.5249 - val_binary_accuracy: 0.7368
Epoch 4/50
16/16 [==============================] - 5s 213ms/step - loss: 0.5042 - binary_accuracy: 0.7695 - val_loss: 0.4709 - val_binary_accuracy: 0.7770
Epoch 5/50
16/16 [==============================] - 5s 211ms/step - loss: 0.4729 - binary_accuracy: 0.7761 - val_loss: 0.4596 - val_binary_accuracy: 0.7870
Epoch 6/50
16/16 [==============================] - 5s 223ms/step - loss: 0.4511 - binary_accuracy: 0.7904 - val_loss: 0.4568 - val_binary_accuracy: 0.7985
Epoch 7/50
16/16 [==============================] - 5s 212ms/st

In [27]:
np.argmin(h1.history["val_loss"]), np.min(h1.history["val_loss"])

(21, 0.33883190155029297)

In [28]:
np.argmax(h1.history["val_binary_accuracy"]), np.max(h1.history["val_binary_accuracy"])

(45, 0.875124990940094)

# Load best frozen weights before fine tuning

In [32]:
tl_model.load_weights(mcp_best_frozen_weights_file)

# Fine tune training

In [33]:
# Make base model trainable (leave layers in inference mode)
models.toggle_TL_freeze(tl_model)

In [34]:
# tl_model.compile(optimizer=Adam(learning_rate=fine_tune_lr), loss=BinaryCrossentropy(), metrics=[BinaryAccuracy()])
tl_model.compile(
    optimizer=Adam(learning_rate=fine_tune_lr, beta_1=adam_beta_1, beta_2=adam_beta_2),
    loss=BinaryCrossentropy(),
    weighted_metrics=[BinaryAccuracy()]
)

In [35]:
tl_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
base_model (Functional)      (None, 4, 4, 2048)        14644096  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 2049      
_________________________________________________________________
activation (Activation)      (None, 1)                 0         
Total params: 14,646,145
Trainable params: 14,605,313
Non-trainable params: 40,832
_________________________________________________________________


In [36]:
es_callback = EarlyStopping(monitor=es_criterion, mode=es_mode, min_delta=es_min_delta, patience=es_patience)
mcp_callback = ModelCheckpoint(mcp_best_finetune_weights_file, monitor=mcp_criterion, mode=mcp_mode, save_best_only=mcp_best_only, save_weights_only=mcp_weights_only)

In [37]:
h2 = tl_model.fit(
    train_datagen,
    validation_data=val_datagen,
    epochs=fine_tune_epochs,
    callbacks=[es_callback, mcp_callback],
    workers=n_cores
)

Epoch 1/50
16/16 [==============================] - 14s 335ms/step - loss: 1.1890 - binary_accuracy: 0.5652 - val_loss: 0.5190 - val_binary_accuracy: 0.7384
Epoch 2/50
16/16 [==============================] - 6s 248ms/step - loss: 0.4982 - binary_accuracy: 0.7490 - val_loss: 0.4343 - val_binary_accuracy: 0.8547
Epoch 3/50
16/16 [==============================] - 6s 269ms/step - loss: 0.4033 - binary_accuracy: 0.8098 - val_loss: 0.6347 - val_binary_accuracy: 0.7261
Epoch 4/50
16/16 [==============================] - 5s 242ms/step - loss: 0.4650 - binary_accuracy: 0.7721 - val_loss: 0.4048 - val_binary_accuracy: 0.8044
Epoch 5/50
16/16 [==============================] - 6s 250ms/step - loss: 0.3701 - binary_accuracy: 0.8189 - val_loss: 0.3657 - val_binary_accuracy: 0.8269
Epoch 6/50
16/16 [==============================] - 6s 242ms/step - loss: 0.3643 - binary_accuracy: 0.8358 - val_loss: 0.3299 - val_binary_accuracy: 0.8243
Epoch 7/50
16/16 [==============================] - 6s 250ms/st

In [38]:
np.argmin(h2.history["val_loss"]), np.min(h2.history["val_loss"])

(32, 0.28118646144866943)

In [39]:
np.argmax(h2.history["val_binary_accuracy"]), np.max(h2.history["val_binary_accuracy"])

(30, 0.8999000191688538)